In [1]:
# run sentiment analysis for OpenAI API and Azure API

from utils import *
from tqdm import tqdm
import pandas as pd 
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI,AzureOpenAI

def inference(prompt, client, model):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}], 
            max_tokens=4,
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        print("!!!!! Error when processing prompt !!!!!")
        print(prompt)
        # print(f"!!!!! {e} !!!!!")
        return None

def batch_inference(batch_data, client, model, max_workers=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = list(
            tqdm(
                executor.map(lambda x: inference(x, client, model), batch_data),
                total=len(batch_data)
            )
        )
        for future in futures:
            results.append(future)
    return results

In [3]:
dataset="FIQASA"
# dataset="FPB"
test_dataset = TestDataset(dataset=dataset)

print("----- Load Dataset -----")
test_dataset = TestDataset(dataset=dataset)
batch_data = [test_dataset[i]['sentence'] for i in range(len(test_dataset))]

print("----- Inference Deepseek -----")
deepseek = OpenAI(api_key="sk-619b8461a6e546d5a69c207e9b37ddc2", base_url="https://api.deepseek.com")
deepseek_outputs = batch_inference(batch_data, deepseek, "deepseek-chat", 16)
save_to_csv(dataset_map[dataset]['path'], "deepseek", deepseek_outputs)

print("----- Inference Azure GPT-4O-Mini -----")
gpt = AzureOpenAI(api_key="b9135a15c242432cb20ddc43fea3a413", api_version="2023-06-01-preview", azure_endpoint="https://openai-oe.openai.azure.com/")
gpt_outputs = batch_inference(batch_data, gpt, "gpt-4o-mini", 16)
save_to_csv(dataset_map[dataset]['path'], "openai", gpt_outputs)

----- Load Dataset -----
----- Inference Deepseek -----


  0%|          | 0/235 [00:00<?, ?it/s]

100%|██████████| 235/235 [00:16<00:00, 14.31it/s]


Save to data/sentimental_analysis/FIQASA/test.csv
----- Inference Azure GPT-4O-Mini -----


  0%|          | 1/235 [00:01<04:40,  1.20s/it]

...

!!!!! Error when processing prompt !!!!!
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive} without anything else.
Input: $YANG Double bottom 5.2
Answer: 


 31%|███       | 73/235 [00:06<00:09, 17.89it/s]

!!!!! Error when processing prompt !!!!!
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive} without anything else.
Input: $GOOG resistance turned support?  It
Answer: 
!!!!! Error when processing prompt !!!!!
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive} without anything else.
Input: $PKT Momentum building for breakout trigger ! Long setup http://stks.co/eXVU
Answer: 


100%|██████████| 235/235 [00:13<00:00, 18.00it/s]

Save to data/sentimental_analysis/FIQASA/test.csv
